In [1]:
# Environment and GPU sanity checks
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # choose GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
print('Python:', sys.version)
print('TF version:', tf.__version__)
print('Physical GPUs:', tf.config.list_physical_devices('GPU'))
# Enable memory growth early
for g in tf.config.list_physical_devices('GPU'):
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception as e:
        print('Memory growth warning:', e)
# Print device placement to confirm GPU usage
tf.debugging.set_log_device_placement(True)


2025-09-13 03:24:49.598437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757721289.621185 1619395 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757721289.628472 1619395 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757721289.645279 1619395 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757721289.645302 1619395 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757721289.645304 1619395 computation_placer.cc:177] computation placer alr

Python: 3.10.12 (main, Aug 15 2025, 14:32:43) [GCC 11.4.0]
TF version: 2.19.0
Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Build config with stronger defaults for GPU utilization
from myxtts.config.config import XTTSConfig, ModelConfig, DataConfig, TrainingConfig
from myxtts.utils.performance import start_performance_monitoring
start_performance_monitoring()

# Dataset paths
train_data_path = '../dataset/dataset_train'
val_data_path = '../dataset/dataset_eval'
print('Train path exists:', os.path.exists(train_data_path))
print('Val path exists  :', os.path.exists(val_data_path))

# Tunables
BATCH_SIZE = 32  # increase if VRAM allows (e.g., 48)
NUM_WORKERS = max(4, (os.cpu_count() or 8)//2)

m = ModelConfig(
    text_encoder_dim=256,
    decoder_dim=512,
    n_mels=80,
    use_voice_conditioning=True
)
t = TrainingConfig(
    epochs=200,
    learning_rate=5e-5,
    optimizer='adamw',
    warmup_steps=2000,
    multi_gpu=False,
    visible_gpus=None,
    log_step=100
)
d = DataConfig(
    batch_size=BATCH_SIZE,
    metadata_train_file='metadata_train.csv',
    metadata_eval_file='metadata_eval.csv',
    wavs_train_dir='wavs',
    wavs_eval_dir='wavs',
    sample_rate=22050,
    normalize_audio=True,
    num_workers=NUM_WORKERS,
    enable_memory_mapping=True,
    prefetch_buffer_size=8,
    shuffle_buffer_multiplier=20,
    max_mel_frames=512,
    enable_xla=True,
    mixed_precision=True
)

config = XTTSConfig(model=m, data=d, training=t)
print('Batch size:', config.data.batch_size, '| Workers:', config.data.num_workers)


Performance monitoring started
Train path exists: True
Val path exists  : True
Batch size: 32 | Workers: 32


In [3]:
# Optional: one-time cache precompute to remove CPU/I-O bottlenecks
PRECOMPUTE = True
if PRECOMPUTE:
    from myxtts.data.ljspeech import LJSpeechDataset
    print('Precomputing caches...')
    ds_tr = LJSpeechDataset(train_data_path, config.data, subset='train', download=False, preprocess=True)
    ds_va = LJSpeechDataset(val_data_path,   config.data, subset='val',   download=False, preprocess=True)
    ds_tr.precompute_mels(num_workers=config.data.num_workers, overwrite=False)
    ds_va.precompute_mels(num_workers=config.data.num_workers, overwrite=False)
    ds_tr.precompute_tokens(num_workers=config.data.num_workers, overwrite=False)
    ds_va.precompute_tokens(num_workers=config.data.num_workers, overwrite=False)
    print('Verifying caches...')
    print('Train verify:', ds_tr.verify_and_fix_cache(fix=True))
    print('Val verify  :', ds_va.verify_and_fix_cache(fix=True))
    print('Train usable:', ds_tr.filter_items_by_cache())
    print('Val usable  :', ds_va.filter_items_by_cache())
    del ds_tr, ds_va


Precomputing caches...
Loaded 20509 items for train subset
Loaded 2591 items for val subset
Precomputing mel spectrograms to ../dataset/dataset_train/processed/mels_sr22050_n80_hop256 (overwrite=False)...
All mel spectrograms already cached.
Precomputing mel spectrograms to ../dataset/dataset_eval/processed/mels_sr22050_n80_hop256 (overwrite=False)...
All mel spectrograms already cached.
Verifying caches...
Train verify: {'checked': 20509, 'fixed': 0, 'failed': 0}
Val verify  : {'checked': 2591, 'fixed': 0, 'failed': 0}
Train usable: 20509
Val usable  : 2591


In [4]:
# Training with GPU monitoring and timing
from myxtts import get_xtts_model, get_trainer, get_inference_engine
from gpu_monitor import GPUMonitor

model = get_xtts_model()(config.model)
trainer = get_trainer()(config, model)
train_dataset, val_dataset = trainer.prepare_datasets(train_data_path=train_data_path, val_data_path=val_data_path)
print('Train samples:', getattr(trainer, 'train_dataset_size', 'n/a'))
print('Val samples  :', getattr(trainer, 'val_dataset_size', 'n/a'))

monitor = GPUMonitor(interval=0.5, log_to_file=False)
monitor.start_monitoring()
try:
    trainer.train(train_dataset, val_dataset)
finally:
    monitor.stop_monitoring()
    print(monitor.get_summary_report())


input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0

I0000 00:00:1757721302.517360 1619395 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22135 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-09-13 03:25:02.533201: I tensorflow/core/common_runtime/placer.cc:162] input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.533219: I tensorflow/core/common_runtime/placer.cc:162] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.533222: I tensorflow/core/common_runtime/placer.cc:162] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.541606: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.541620: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.542544: I tens

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/devic

2025-09-13 03:25:02.742297: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.742306: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.746480: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.746489: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.749404: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.749413: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:02.754313: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /j

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/devic

...

2025-09-13 03:25:22.303575: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:22.303591: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:22.304748: I tensorflow/core/common_runtime/placer.cc:162] resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:22.304755: I tensorflow/core/common_runtime/placer.cc:162] value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:22.304760: I tensorflow/core/common_runtime/placer.cc:162] AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:22.307615: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:22.307623: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job

2025-09-13 03:25:23.301639: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:23.301658: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:23.303023: I tensorflow/core/common_runtime/placer.cc:162] resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:23.303033: I tensorflow/core/common_runtime/placer.cc:162] value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:23.303040: I tensorflow/core/common_runtime/placer.cc:162] AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:23.305157: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-09-13 03:25:23.305168: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/

GPU monitoring stopped
No GPU metrics collected


AttributeError: in user code:

    File "/home/dev371/xTTS/MyXTTSModel/myxtts/training/trainer.py", line 333, in train_step_fn  *
        return self.train_step(text_sequences, mel_spectrograms, text_lengths, mel_lengths)
    File "/home/dev371/xTTS/MyXTTSModel/myxtts/training/trainer.py", line 406, in train_step  *
        scaled_loss = self.optimizer.get_scaled_loss(scaled_loss)

    AttributeError: 'AdamW' object has no attribute 'get_scaled_loss'


In [ ]:
# Inference demo
from myxtts import get_inference_engine
inference = get_inference_engine()(config, checkpoint_path='./checkpoints/best')
result = inference.synthesize('Hello world!')
inference.save_audio(result['audio'], 'output.wav')
